In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision import models,datasets
from tensorboardX import SummaryWriter

In [2]:
cat_img=Image.open('1280px-Felis_silvestris_catus_lying_on_rice_straw.jpg')
cat_img.size

(1280, 853)

In [3]:
# 这是一张张280×853的图，我们先把它变成224×224的图片，便于VGG16使用
transform_224=transforms.Compose([
    #cmpose 组成
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
cat_img_224=transform_224(cat_img)

In [7]:
# 将图片在tensorboard中展示
writer=SummaryWriter(log_dir='./logs',comment="cat_img")
# 这里的logs要与--logdir的参数一样
writer.add_image('cat',cat_img_224)
writer.close() # 执行close立即刷新，否则将每张着秒自动刷新

In [6]:
# 更新损失函数
x=torch.FloatTensor([100])
y=torch.FloatTensor([500])
for epoch in range(100):
    x/=1.5
    y/=1.5
    loss=y-x
    with SummaryWriter(log_dir='./logs',comment='train') as writer:
        # 可以直接使用python的with语法，自动调用close方法
        writer.add_histogram('his/x',x,epoch)
        writer.add_histogram('his/y',y,epoch)
        writer.add_scalar('data/x',x,epoch)
        writer.add_scalar('data/y',y,epoch)
        writer.add_scalar('data/loss',loss,epoch)
        writer.add_scalars('data/data_group',{'x':x,
                                             'y':y,
                                             'loss':loss},epoch)

In [2]:
BATCH_SIZE=256
EPOCHS=20
train_loader=torch.utils.data.DataLoader(datasets.MNIST('MNIST',
                                                        train=True,
                                                        download=False,
                                                        transform=transforms.Compose([
                                                            transforms.ToTensor(),
                                                            transforms.Normalize((0.1307,),(0.3081,)),
                                                        ])),
                                         batch_size=BATCH_SIZE,
                                         shuffle=True
                                        )

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28*28
        self.conv1=nn.Conv2d(1,10,5) #10,24*24
        self.conv2=nn.Conv2d(10,20,3)
        self.fc1=nn.Linear(20*10*10,500)
        self.fc2=nn.Linear(500,10)
    def forward(self,x):
        in_size=x.size(0)
        out=self.conv1(x) #24
        out=F.relu(out)
        out=F.max_pool2d(out,2,2) #12
        out=self.conv2(out)
        out=F.relu(out)
        out=out.view(in_size,-1)
        out=self.fc1(out)
        out=F.relu(out)
        out=self.fc2(out)
        out=F.log_softmax(out,dim=1)
        return out
model=ConvNet()
optimizer=torch.optim.Adam(model.parameters())

In [4]:
def train(model,train_loader,optimizer,epoch):
    n_iter=0
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        optimizer.zero_grad()
        output=model(data)
        loss=F.nll_loss(output,target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30==0:
            n_iter+=1
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch,batch_idx*len(data),len(train_loader.dataset),
            100.*batch_idx/len(train_loader),loss.item()))
            # 增加内容
            out=torch.cat((output.data,torch.ones(len(output),1)),1)
            # 因为投影到3D空间，所以只需要3个维度
            with SummaryWriter(log_dir='./logs',comment='MNIST') as writer:
                writer.add_embedding(
                out,
                metadata=target.data,
                label_img=data.data,
                global_step=n_iter)

In [ ]:
train(model,train_loader,optimizer,i)

In [3]:
# 绘制网络结构，由于pytorch使用的是动态图计算，所以需要手动向前传播一次
vgg16=models.vgg16(pretrained=True) # 下载预训练的网络模型
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [4]:
# 在向前传播前，先把图片做一些调整
transform_2=transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
])

In [5]:
vgg16_input=transform_2(cat_img)[np.newaxis] 
# 因为pytorch是分批次进行的，所以我们建立一个批次为1的数据集
vgg16_input.shape

torch.Size([1, 3, 224, 224])

In [8]:
# 开始向前传播，打印输出值
out=vgg16(vgg16_input)
_,preds=torch.max(out.data,1)
label=preds.numpy()[0]
label

277

In [7]:
with SummaryWriter(log_dir='./logs', comment='vgg16') as writer:
    writer.add_graph(vgg16,(vgg16_input,))

AssertionError: Only output_size=[1, 1] is supported